In [11]:
import pandas as pd

In [12]:
# read datast(s)
with open('../data/Coaches.csv', 'rb') as f:
    coaches = pd.read_csv(f)

with open('../data/Stadium.csv', 'rb') as f:
    stadium = pd.read_csv(f)

with open('../data/2018RES_File1-DI-NationalAggregation-OverallSA_Cohort9510_v1_20180213.csv', 'rb') as f:
    combined = pd.read_csv(f)

with open('../data/2018RES_File2-DI-NationalAggregation-SAgenderrace_Cohort9510_v1_20180213.csv', 'rb') as f:
    athletes_only = pd.read_csv(f)

with open('../data/2018RES_File3-DI-NationalAggregation-SB_Coh9510_v1_20180213.csv', 'rb') as f:
    students_only = pd.read_csv(f)

In [13]:
print(coaches)

                    School Conference               Coach    SchoolPay  \
0                Air Force   Mt. West        Troy Calhoun       885000   
1                    Akron        MAC        Terry Bowden    $411,000    
2                  Alabama        SEC          Nick Saban  $8,307,000    
3    Alabama at Birmingham      C-USA          Bill Clark    $900,000    
4        Appalachian State   Sun Belt   Scott Satterfield    $712,500    
5                  Arizona     Pac-12        Kevin Sumlin  $1,600,000    
6            Arizona State     Pac-12        Herm Edwards  $2,000,000    
7                 Arkansas        SEC         Chad Morris  $3,500,000    
8           Arkansas State   Sun Belt      Blake Anderson    $825,000    
9                     Army       Ind.         Jeff Monken       932521   
10                  Auburn        SEC         Gus Malzahn  $6,700,000    
11              Ball State        MAC            Mike Neu    $435,689    
12                  Baylor     Big 12 